# Import library

In [1]:
import importlib
import torch
import pickle
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pNN_Split as pnn
import random
import config

# Random Seed

In [2]:
seed = 0

# Device

In [3]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda:0')
device = 'cpu'
device

'cpu'

# Prepare data

## Datasets

In [4]:
# if config.current_dataset is not None:
#     ds = config.datasets[config.current_dataset]
# else:
#     ds = config.datasets[0]

# ds

In [5]:
ds1 = 'Seeds'

In [6]:
ds2 = 'Iris'

## Load data

In [7]:
datapath = os.path.join(os.getcwd(), 'Datasets',
                        'dataset_processed', f'Dataset_{ds1}.p')
with open(datapath, 'rb') as f:
    dataset = pickle.load(f)
X1 = dataset['X'].float()
y1 = dataset['y']

E1, N_features1, N_class1 = X1.shape[0], X1.shape[1], torch.max(torch.unique(y1)).item()+1
X1.shape, y1.shape, E1, N_features1, N_class1

(torch.Size([210, 7]), torch.Size([210]), 210, 7, 3)

In [8]:
datapath = os.path.join(os.getcwd(), 'Datasets',
                        'dataset_processed', f'Dataset_{ds2}.p')
with open(datapath, 'rb') as f:
    dataset = pickle.load(f)
X2 = dataset['X'].float()
y2 = dataset['y']

E2, N_features2, N_class2 = X2.shape[0], X2.shape[1], torch.max(torch.unique(y2)).item()+1
X2.shape, y2.shape, E2, N_features2, N_class2

(torch.Size([150, 4]), torch.Size([150]), 150, 4, 3)

## Pseudo-electrical Signal

In [9]:
X1 = X1 - torch.min(X1, axis=0)[0]
X1 = X1 / (torch.max(X1, axis=0)[0])
torch.min(X1), torch.max(X1)

(tensor(0.), tensor(1.))

In [10]:
X2 = X2 - torch.min(X2, axis=0)[0]
X2 = X2 / (torch.max(X2, axis=0)[0])
torch.min(X2), torch.max(X2)

(tensor(0.), tensor(1.))

In [11]:
# split
train_rate = 0.6
valid_rate = 0.2
test_rate = 0.2

E_train1 = min(8192, int(E1*train_rate))
E_valid1 = min(2048, int(E1*valid_rate))
E_test1 = min(2048, int(E1*test_rate))

random.seed(config.data_split_seed);
np.random.seed(config.data_split_seed);
torch.manual_seed(config.data_split_seed);

idx = torch.randperm(E1)
X1 = X1[idx,:]
y1 = y1[idx]

X_train1, X_rest1 = X1[:E_train1,:],      X1[E_train1:,:]
X_valid1, X_rest1 = X_rest1[:E_valid1,:], X_rest1[E_valid1:,:]
X_test1,  X_rest1 = X_rest1[:E_test1,:],  X_rest1[E_test1:,:]

y_train1, y_rest1 = y1[:E_train1],      y1[E_train1:]
y_valid1, y_rest1 = y_rest1[:E_valid1], y_rest1[E_valid1:]
y_test1,  y_rest1 = y_rest1[:E_test1],  y_rest1[E_test1:]

X_train1.shape, y_train1.shape, X_valid1.shape, y_valid1.shape, X_test1.shape, y_test1.shape

(torch.Size([126, 7]),
 torch.Size([126]),
 torch.Size([42, 7]),
 torch.Size([42]),
 torch.Size([42, 7]),
 torch.Size([42]))

In [12]:
E_train2 = min(8192, int(E2*train_rate))
E_valid2 = min(2048, int(E2*valid_rate))
E_test2 = min(2048, int(E2*test_rate))

random.seed(config.data_split_seed);
np.random.seed(config.data_split_seed);
torch.manual_seed(config.data_split_seed);

idx = torch.randperm(E2)
X2 = X2[idx,:]
y2 = y2[idx]

X_train2, X_rest2 = X2[:E_train2,:],      X2[E_train2:,:]
X_valid2, X_rest2 = X_rest2[:E_valid2,:], X_rest2[E_valid2:,:]
X_test2,  X_rest2 = X_rest2[:E_test2,:],  X_rest2[E_test2:,:]

y_train2, y_rest2 = y2[:E_train2],      y1[E_train2:]
y_valid2, y_rest2 = y_rest2[:E_valid2], y_rest2[E_valid2:]
y_test2,  y_rest2 = y_rest2[:E_test2],  y_rest2[E_test2:]

X_train2.shape, y_train2.shape, X_valid2.shape, y_valid2.shape, X_test2.shape, y_test2.shape

(torch.Size([90, 4]),
 torch.Size([90]),
 torch.Size([30, 4]),
 torch.Size([30]),
 torch.Size([30, 4]),
 torch.Size([30]))

In [13]:
X_trains = [X_train1, X_train2]
y_trains = [y_train1, y_train2]

In [14]:
X_valids = [X_valid1, X_valid2]
y_valids = [y_valid1, y_valid2]

In [15]:
X_tests = [X_test1, X_test2]
y_tests = [y_test1, y_test2]

# PNN

In [16]:
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);

## Define

In [17]:
num_in = []
num_in.append(N_features1)
num_in.append(N_features2)
num_in

[7, 4]

In [18]:
num_out = []
num_out.append(N_class1)
num_out.append(N_class2)
num_out

[3, 3]

In [19]:
topology_hidden = [3, 3]

In [20]:
SuperPNN = pnn.PNNSupernet(num_in, num_out, topology_hidden)

In [21]:
optimizer = torch.optim.Adam([SuperPNN.hidden_theta], lr=0.1)

In [22]:
loss_train = []
loss_valid = []
for epoch in range(1000):
    optimizer.zero_grad()
    
    predictions_train = SuperPNN.forward(X_trains)
#     L_train = pnn.LOSSFUNCTION(predictions_train, y_trains)
    L_train = 0
    for p in predictions_train:
        L_train = L_train + p.sum()
    L_train.backward()
    optimizer.step()
    
    with torch.no_grad():
        predictions_valid = SuperPNN.forward(X_valids)
        L_valid = pnn.LOSSFUNCTION(predictions_valid, y_valids)
    
    loss_train.append(L_train.data)
    loss_valid.append(L_valid.data)
    
    if not epoch%100:
        print(L_train.data, L_valid.data)

tensor(710.2079) tensor(2.7920)
tensor(710.2079) tensor(2.7920)
tensor(710.2079) tensor(2.7920)
tensor(710.2079) tensor(2.7920)
tensor(710.2079) tensor(2.7920)
tensor(710.2079) tensor(2.7920)
tensor(710.2079) tensor(2.7920)
tensor(710.2079) tensor(2.7920)
tensor(710.2079) tensor(2.7920)
tensor(710.2079) tensor(2.7920)


In [23]:
L_train

tensor(710.2079, grad_fn=<AddBackward0>)

In [24]:
for n,p in SuperPNN.named_parameters():
    print(n, p.grad)

hidden_theta None
models.0.model.Input_Layer.theta_ tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
models.0.model.Hiddel_Layer 0.theta_ tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
models.0.model.Output_Layer.theta_ tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
models.1.model.Input_Layer.theta_ tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
models.1.model.Hiddel_Layer 0.theta_ tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
models.1.model.Output_Layer.theta_ tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 

In [25]:
SuperPNN.hidden_theta

Parameter containing:
tensor([[4.9626e-01, 7.6822e-01],
        [8.8477e-02, 1.3203e-01],
        [3.0742e-01, 6.3408e-01],
        [4.9009e-01, 8.9644e-01],
        [4.5563e-01, 6.3231e-01],
        [3.4889e-01, 4.0172e-01],
        [2.2326e-02, 1.6886e-01],
        [2.9389e-01, 5.1852e-01],
        [6.9767e-01, 8.0001e-01],
        [9.3465e+00, 1.2401e+01],
        [2.0862e+01, 9.5384e-01],
        [4.1759e+00, 8.3559e+00],
        [4.1941e+01, 5.5291e+01],
        [9.5274e+01, 3.6165e+00],
        [1.8523e+01, 3.7342e+01]], requires_grad=True)

In [26]:
SuperPNN.hidden_theta.grad

In [27]:
kkk = SuperPNN.hidden_theta.sum()

In [28]:
kkk.backward()

In [29]:
SuperPNN.hidden_theta.grad

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

## Training

In [30]:
optimizer_PNN = torch.optim.Adam(PNN.parameters(), lr=config.lr)
train_loader = DataLoader(train_data, batch_size=len(train_data))
train_loss_PNN, valid_loss_PNN, parameter_PNN = tv.train_normal_pnn(PNN, train_loader, valid_loader,
                                                                   config.m, config.T, optimizer_PNN,
                                                                   pnnv.LossFunction, Epoch=50000, cache=ds)

NameError: name 'PNN' is not defined

In [ ]:
plt.plot(train_loss_PNN, '-', label='train')
plt.plot(valid_loss_PNN, '--', label='valid')
plt.legend()
best = np.argmin(valid_loss_PNN)
PNN.load_state_dict(parameter_PNN)
best, valid_loss_PNN[best]

## Save Model

In [ ]:
with open(f'./result/{ds}_PNN_{seed}.p', 'wb') as f:
    pickle.dump(PNN, f)